<a href="https://colab.research.google.com/github/ElnazDi/colab/blob/master/Feature_Engineering_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from google.colab import files
data = files.upload()

In [48]:
import pandas as pd
data = pd.read_csv('ks-projects-201801.csv', parse_dates=['deadline', 'launched'])
data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0.0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15.0,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3.0,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1.0,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14.0,US,1283.0,1283.0,19500.00


In [49]:
# Set the target/ Y column
y = data['state']
y = pd.unique(y)
y

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended', nan], dtype=object)

In [50]:
# count the unique values in state column (same with bottom)
data.groupby('state').ID.count()

state
canceled      1063
failed        5561
live            90
successful    3789
suspended       46
undefined      104
Name: ID, dtype: int64

In [51]:
# count the unique values in state column (same with top)
data.state.value_counts()

failed        5561
successful    3789
canceled      1063
undefined      104
live            90
suspended       46
Name: state, dtype: int64

In [52]:
# dropping the live videos by getting query between sate coulmn if they are not live keep it
data= data.query('state != "live"')
drop_live= pd.unique(data['state'])
drop_live

array(['failed', 'canceled', 'successful', 'undefined', 'suspended', nan],
      dtype=object)

In [53]:
# assigning new coloumn called outcome --- its entry is 0/1 but not boolean as integer
data = data.assign(outcome=(data['state']== 'successful').astype(int))
data.tail()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome
10649,1053521466,Support High School Dance Team,Performances,Dance,USD,2016-04-05,10000.0,2016-03-21 17:04:39,15.0,failed,1.0,US,15.0,15.0,10000.0,0
10650,1053529450,ETL:Survival,Video Games,Games,USD,2016-05-29,100000.0,2016-03-30 01:48:54,60.0,failed,3.0,US,60.0,60.0,100000.0,0
10651,1053538584,SWYK - Sharing Wisdom & Your Knowledge Campaign,Apps,Technology,USD,2016-10-23,35000.0,2016-09-13 00:12:32,5731.0,failed,50.0,US,2368.0,5731.0,35000.0,0
10652,1053538918,400 Miles Away - EP by Chase Miller (Canceled),Pop,Music,USD,2013-04-05,1300.0,2013-03-06 09:09:16,0.0,canceled,0.0,US,0.0,0.0,1300.0,0
10653,1053542395,Dj Pope & Marc Evans - Live It Up,Music Videos,Film & Video,GBP,2014-08-13,350.0,2014-07-30 20:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [54]:
# assigning 4 extra column to data frame (hour,day,month,year) with .dt (from launched coulumn which is date type)

data = data.assign( hour = data.launched.dt.hour,
                    day = data.launched.dt.day,
                    month = data.launched.dt.month,
                    year= data.launched.dt.year)
data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0.0,GB,0.0,0.0,1533.95,0,12,11,8,2015
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15.0,US,100.0,2421.0,30000.00,0,4,2,9,2017
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3.0,US,220.0,220.0,45000.00,0,0,12,1,2013
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1.0,US,1.0,1.0,5000.00,0,3,17,3,2012
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14.0,US,1283.0,1283.0,19500.00,0,8,4,7,2015


In [56]:
# Now working with features (Country, Currency, Category) --- These are categorical features which 
# We want to convert them to integers for our model
# We need a new library for preprocessing

from sklearn.preprocessing import LabelEncoder

cat_features = ['category', 'currency']   #I can not add country I do not know what is the problem
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded = data[cat_features].apply(encoder.fit_transform)
encoded.head(10)

,category,currency
0,106,5
1,91,13
2,91,13
3,88,13
4,54,13
5,120,13
6,57,13
7,40,13
8,111,13
9,38,13


In [57]:
new_data = data[['goal', 'hour', 'day', 'month','year', 'outcome']].join(encoded)
new_data.head()

,goal,hour,day,month,year,outcome,category,currency
0,1000.0,12,11,8,2015,0,106,5
1,30000.0,4,2,9,2017,0,91,13
2,45000.0,0,12,1,2013,0,91,13
3,5000.0,3,17,3,2012,0,88,13
4,19500.0,8,4,7,2015,0,54,13


In [0]:
#A good way to do this automatically is with sklearn.model_selection.StratifiedShuffleSplit but I don't need to use it here.
#We'll use 10% of the data as a validation set, 10% for testing, and the other 80% for training.
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

train = new_data[:-2 * valid_size]
valid = new_data[-2 * valid_size:-valid_size]
test = new_data[-valid_size:]

In [59]:
for each in [train, valid, test]:
    print(f"Outcome fraction = {each.outcome.mean():.4f}")

Outcome fraction = 0.3600
Outcome fraction = 0.3589
Outcome fraction = 0.3475


In [0]:
#Training a LightGBM model

import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, verbose_eval=False)

In [61]:
#Making predictions & evaluating the model

from sklearn import metrics
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)

print(f"Test AUC score: {score}")


Test AUC score: 0.7303381673079888
